<a href="https://colab.research.google.com/github/hegelty/hate-classifier/blob/main/%ED%98%90%EC%98%A4%ED%91%9C%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers[torch]
!pip install lime

In [16]:
import csv
import pandas as pd
import numpy as np

In [3]:
# 데이터셋 로드
train_file = "./unsmile_train_v1.0.tsv"
train_dataset = []
with open(train_file, 'r') as f:
    tr = csv.reader(f, delimiter='\t')
    for row in tr:
        train_dataset.append(row)

val_file = "./unsmile_valid_v1.0.tsv"
val_dataset = []
with open(val_file, 'r') as f:
    tr = csv.reader(f, delimiter='\t')
    for row in tr:
        val_dataset.append(row)
        
        
print(len(train_dataset), len(val_dataset))
print(val_dataset[1:4])

15006 3738
[['ㅇㄱㄹㅇ 진짜 죽어도 상관없다는 마인드로 싸웠더니 지금 서열 상타취노 식칼들고 니가 나 안찌르면 내가 너 찌른다 했더니 애비충 냄동충 알아서기노 ^됫^ 한번 서열 잡고 그 담에 개길때마다 더 세게나가면 확실하게 짓누를수있다익이', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['여자들은 취미가 애낳는건가.. 취미를 좀 가져라', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['개슬람녀 다 필요없고 니 엄마만 있으면 된다', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0']]


In [4]:
from transformers import AutoModelForSequenceClassification

model_checkpoint = "beomi/KcBERT-Base"  # 사용할 모델
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)  # 모델 불러오기

Some weights of the model checkpoint at beomi/KcBERT-Base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)  # 토크나이져 가져오기

In [6]:
from tqdm.notebook import tqdm
import datasets

# 데이터셋 정제
dataset = datasets.DatasetDict({
    "train": datasets.Dataset.from_list([
          {
            "text": data[0],
            "label": int(data[10] or data[11]) # 혐오표현 여부(0: 험오표현, 1: 비험오표현)
          } for data in train_dataset[1:]
        ]),
    "test": datasets.Dataset.from_list([
          {
            "text": data[0],
            "label": int(data[10] or data[11])
          } for data in val_dataset[1:]
        ]),
})

print(dataset["train"][:3])

{'text': ['일안하는 시간은 쉬고싶어서 그런게 아닐까', '아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. 10 16까지 IQ 떨어지고 출산 위험은 400% 다.', '루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o  doin 진짜 띵곡임 꼭 들어보셈"'], 'label': [1, 0, 1]}


In [7]:
def tokenize_function(examples):  # 토크나이저 함수
    return tokenizer(examples["text"], truncation=True)

# 토크나이징
tokenized_dataset = dataset.map(tokenize_function,
                                 batched=True,  # 배치
                                 num_proc=4,  # 멀티 프로세싱
                                 remove_columns=["text"])  # text 열 제거

Map (num_proc=4):   0%|          | 0/15005 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3737 [00:00<?, ? examples/s]

In [8]:
print(tokenized_dataset["train"][0])

{'label': 1, 'input_ids': [2, 2458, 15751, 24930, 24351, 29278, 17038, 11631, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [17]:
# train 데이터셋을 데이터프레임으로 변환
train_data = dataset['train'][:]
train_df = pd.DataFrame(train_data)

# test 데이터셋을 데이터프레임으로 변환
test_data = dataset['test'][:]
test_df = pd.DataFrame(test_data)

combined_df = pd.concat([train_df, test_df], ignore_index=True)
combined_df

,text,label
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,1
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,1
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,1
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,0
...,...,...
18737,저게 시대적언어면 한남충도 시대적언어 아니노 ㅋㅋ,0
18738,다른것보다 눈이 어떻게 저렇게 생기노.탄식.,0
18739,막노동을 해도 한국에살고말지 미쳤다고 남미를가냐?차라리 자살을하겠다.,0
18740,‘사형을 구형하였으나 여성인 점을 감안해 25년 선고’ ???내가 뭐 잘못본건가?개...,0


In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)  # 패딩

In [10]:
from huggingface_hub import notebook_login
notebook_login()  # 허깅페이스 로그인

In [18]:
from transformers import TextClassificationPipeline, BertForSequenceClassification, AutoTokenizer
model_name = 'beomi/KcBERT-Base'

model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = TextClassificationPipeline(
    model = model,
    tokenizer = tokenizer,
    device = 0,   # cpu: -1, gpu: gpu number
    return_all_scores = True,
    function_to_apply = 'softmax'  # sigmoid
)

for result in pipe("이래서 여자는 게임을 하면 안된다")[0]:
    print(result)

Some weights of the model checkpoint at beomi/KcBERT-Base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

{'label': 'LABEL_0', 'score': 0.5044817328453064}
{'label': 'LABEL_1', 'score': 0.4955182373523712}


/opt/conda/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(  # 하이퍼파라미터 설정
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(  # 데이터셋과 모델을 트레이너에 넣어줌
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [20]:
trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.349800
1000,0.294000
1500,0.195100
2000,0.168200
2500,0.103900
3000,0.064300
3500,0.039200
4000,0.025500
4500,0.012000


TrainOutput(global_step=4690, training_loss=0.13401803955086258, metrics={'train_runtime': 948.9886, 'train_samples_per_second': 79.058, 'train_steps_per_second': 4.942, 'total_flos': 1916827760506320.0, 'train_loss': 0.13401803955086258, 'epoch': 5.0})

In [21]:
model_name = model_checkpoint.split("/")[-1]
model.push_to_hub(  # 허깅페이스에 업로드
    f"{model_name}-finetuned-hate"
)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shinnew/KcBERT-Base-finetuned-hate/commit/cc1b183497be50637ea5211d93fe2f5c945f0395', commit_message='Upload BertForSequenceClassification', commit_description='', oid='cc1b183497be50637ea5211d93fe2f5c945f0395', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
tokenizer.push_to_hub(
    f"{model_name}-finetuned-hate"
)

CommitInfo(commit_url='https://huggingface.co/shinnew/KcBERT-Base-finetuned-hate/commit/831fefeedfe75054390aa7b2c8fd59a64357a3a9', commit_message='Upload tokenizer', commit_description='', oid='831fefeedfe75054390aa7b2c8fd59a64357a3a9', pr_url=None, pr_revision=None, pr_num=None)

In [40]:
input_text = "미애들 개새키야. 이 ㅅㅂ놈아. 미친 새끼야. 니 말투 그렇게 하면 누가 ㅅㅂ놈아 믿겠냐."  

# Perform inference
result = pipe(input_text)

print(result)

#tag 0: 혐오 
# tag 1: 일반

[[{'label': 'LABEL_0', 'score': 0.9999656677246094}, {'label': 'LABEL_1', 'score': 3.4287510061403736e-05}]]


In [42]:
input_text1 = "저축은행? 너 이름 뭐야?"  

# Perform inference
result1 = pipe(input_text1)

print(result1)


[[{'label': 'LABEL_0', 'score': 0.00035696482518687844}, {'label': 'LABEL_1', 'score': 0.9996429681777954}]]


-----------------------------------------

### Lime package 활용

In [25]:
from lime import lime_tabular, lime_text
from lime.lime_text import LimeTextExplainer
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

import random
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [26]:
categories = combined_df['label'].unique()
combined_df['label'] = pd.Categorical(combined_df['label'])
combined_df['label_code'] = combined_df['label'].cat.codes
class_names = combined_df['label'].unique()

In [27]:
# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    combined_df['text'], combined_df['label'], test_size=0.2, random_state=42
)

In [31]:
# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data)
X_test = vectorizer.transform(test_data)

In [32]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500, random_state=42)
rf.fit(X_train, train_labels)

RandomForestClassifier(n_estimators=500, random_state=42)

In [33]:
pred = rf.predict(X_test)
sklearn.metrics.f1_score(test_labels, pred, average='binary')

0.48567650947553986

In [34]:
c = make_pipeline(vectorizer, rf)

In [35]:
print(c.predict_proba(combined_df.text.values)) 

[[0.59603858 0.40396142]
 [0.93497888 0.06502112]
 [0.298      0.702     ]
 ...
 [0.96095548 0.03904452]
 [0.68507738 0.31492262]
 [0.944      0.056     ]]


In [39]:
input_text

'미애들 개새키야. 이 ㅅㅂ놈아. 미친 새끼야. 니 말투 그렇게 하면 누가 ㅅㅂ놈아 믿겠냐.'

In [38]:
explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(input_text, c.predict_proba, num_features=6)
print('Probability(욕설) =', c.predict_proba(input_text)[0,1])
print('True class: %s' % class_names[input_text])

# idx = 2
# exp = explainer.explain_instance(combined_df.text.values[idx], c.predict_proba, num_features=6)
# print('Document id: %d' % idx)
# print('Probability(욕설) =', c.predict_proba([combined_df.text.values[idx]])[0,1])
# print('True class: %s' % class_names[combined_df.label.values[idx]])  # 왜 여기 값이 무조건 1이 나오는지 모르겠다.

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 explainer = LimeTextExplainer(class_names=class_names)                                      │
│    2 exp = explainer.explain_instance(input_text, c.predict_proba, num_features=6)               │
│ ❱  3 print('Probability(욕설) =', c.predict_proba(input_text)[0,1])                              │
│    4 print('True class: %s' % class_names[input_text])                                           │
│    5                                                                                             │
│    6 # idx = 2                                                                                   │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/sklearn/utils/metaestimators.py:113 in <lambda>           │
│                                                                                                  │
│   110 │   │   │   │   raise attr_err                                                             │
│   111 │   │   │                                                                                  │
│   112 │   │   │   # lambda, but not partial, allows help() to work with update_wrapper           │
│ ❱ 113 │   │   │   out = lambda *args, **kwargs: self.fn(obj, *args, **kwargs)  # noqa            │
│   114 │   │   else:                                                                              │
│   115 │   │   │                                                                                  │
│   116 │   │   │   def fn(*args, **kwargs):                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py:535 in predict_proba                  │
│                                                                                                  │
│    532 │   │   """                                                                               │
│    533 │   │   Xt = X                                                                            │
│    534 │   │   for _, name, transform in self._iter(with_final=False):                           │
│ ❱  535 │   │   │   Xt = transform.transform(Xt)                                                  │
│    536 │   │   return self.steps[-1][1].predict_proba(Xt, **predict_proba_params)                │
│    537 │                                                                                         │
│    538 │   @available_if(_final_estimator_has("decision_function"))                              │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2093 in transform      │
│                                                                                                  │
│   2090 │   │   """                                                                               │
│   2091 │   │   check_is_fitted(self, msg="The TF-IDF vectorizer is not fitted")                  │
│   2092 │   │                                                                                     │
│ ❱ 2093 │   │   X = super().transform(raw_documents)                                              │
│   2094 │   │   return self._tfidf.transform(X, copy=False)                                       │
│   2095 │                                                                                         │
│   2096 │   def _more_tags(self):                                                                 │
│                                                              

In [ ]:
exp.as_list()

In [ ]:
print('Original prediction:', rf.predict_proba(X_test[idx])[0,1])
tmp = X_test[idx].copy()
# tmp[0,vectorizer.vocabulary_["정도면"]] = 0
# tmp[0,vectorizer.vocabulary_['그냥']] = 0
print('Prediction removing some features:', rf.predict_proba(tmp)[0,1])
print('Difference:', rf.predict_proba(tmp)[0,1] - rf.predict_proba(X_test[idx])[0,1])

In [ ]:
from matplotlib import font_manager, rc


%matplotlib inline

# 한국어 폰트 경로 설정
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

fig = exp.as_pyplot_figure()

In [ ]:
exp.show_in_notebook(text=True)   

------------------------

### Lime 패키지 활용부분 자동화

In [71]:
from lime import lime_tabular, lime_text
from lime.lime_text import LimeTextExplainer
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

import random
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [30]:
def generate_explanation(df, idx, c, class_names, num_features=6):
    # LIME 설명 인스턴스 생성
    explainer = lime_text.LimeTextExplainer(class_names=class_names)    
    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform(train_data)
    X_test = vectorizer.transform(test_data)
    c = make_pipeline(vectorizer, rf)
    
    # LIME 설명 생성
    exp = explainer.explain_instance(df.text.values[idx], c.predict_proba, num_features=num_features)
    pred = rf.predict(X_test)
    

    
    print('Document id:', idx)
    print('Probability(욕설) =', c.predict_proba([df.text.values[idx]])[0, 1])
    print('True class:', class_names[df.label.values[idx]])
    print('F1 Score:', sklearn.metrics.f1_score(test_labels, pred, average='binary')
    
          
# 시각화
fig = exp.as_pyplot_figure()
plt.show()

# 노트북에 텍스트로 표시
exp.show_in_notebook(text=True)

# 사용 예시
idx = 100  # 원하는 인덱스로 설정
generate_explanation(combined_df, idx, c, class_names, num_features=6)

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ fig = exp.as_pyplot_figure()                                                                     │
│ ▲                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax